# Fetching Data

In [2]:
import numpy as np
import pandas as pd
import concurrent.futures
import requests
import time
from functools import partial
from datetime import datetime
import os

In [3]:
def toDatetime(record):
    """
    Convert formatted date-time text into timestamp. 
    It is specifically tailored to the API.
    """
    timestamp = record['timestamp']
    record['timestamp'] = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S")
    return record

def getData(startTime, endTime , freq="Daily"):
    """
    Convert formatted date-time text into timestamp.
    This is specifically tailored to the source of data
    
    Keyword arguments:
    startTime -- the starting date of the period you wish to fetch (YYYY-MM-DD)
    endTime -- the ending date of the period you wish to fetch (YYYY-MM-DD)
    freq -- the frequency of data (Hourly/Daily/...)
    """
    reportingGroups = ["Electricity"]
    locationsLink = 'https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List'
    locationsRequest = requests.get(locationsLink) # initialize api
    locationsRequest.raise_for_status() # get data from api
    locations = pd.DataFrame.from_dict(locationsRequest.json())
    locations = locations['locationName']
    data = []
    dataLink = (f"https://helsinki-openapi.nuuka.cloud/api/v1.0/"
                f"EnergyData/{freq}/ListByProperty")
    
    # Helper function for concurrency 
    def request(group, location):
        payload = {'StartTime': startTime, 'EndTime': endTime}
        payload.update({'ReportingGroup': group})
        payload.update({'SearchString': location.split(' ', 1)[0]})
        r = requests.get(dataLink, payload)
        if r.status_code == requests.codes.ok:
            data_dict = r.json()
            for record in data_dict:
                toDatetime(record)
            data.extend(data_dict)
    
    # The final step - getting the data.
    for g in reportingGroups:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            print(time.time())
            executor.map(partial(request, g), locations)
    # for each "thread", execute "request" function on each location
    return pd.DataFrame(data)


In [107]:
# Request the data from the API.
# Note: You can only make 5000 API requests per hour.
start = time.perf_counter()
ts = getData('2018-01-01', '2019-01-01', freq = 'Hourly') 
finish = time.perf_counter()
print(f'Finished in {finish-start} second(s)')

# Export the Dataframe to a .csv file
file_name = 'hourly2018-2019.csv' # change this to the respective frequency of the data
ts.to_csv(file_name)

1654087579.0468223
Finished in 844.0711271734908 second(s)


In [12]:
data = pd.read_csv('hourly2017-2019.csv')
data.locationName.nunique()

633

In [46]:
data = pd.read_csv('hourly_check.csv')
data = data.drop(['Unnamed: 0', 'reportingGroup', 'unit'],axis=1)
data

,timestamp,locationName,value
0,2017-01-01 00:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",7.89
1,2017-01-01 01:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",7.90
2,2017-01-01 02:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",8.04
3,2017-01-01 03:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",7.93
4,2017-01-01 04:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",8.00
...,...,...,...
28219,2017-01-02 20:00:00,4307 Lp Rudolf,1.07
28220,2017-01-02 21:00:00,4307 Lp Rudolf,1.08
28221,2017-01-02 22:00:00,4307 Lp Rudolf,1.05
28222,2017-01-02 23:00:00,4307 Lp Rudolf,1.05


In [47]:
client_n = 100
cluster_n = min(2, int(data.locationName.nunique()/client_n))

In [48]:
user_name = data.locationName.unique().tolist()
clients = []

for i in range(client_n):
    clients.append(user_name[i*cluster_n:(i+1)*cluster_n]) 
    
clients[:5]

[['1655 Lpk Virkkula, Ruiskumestarin talo', '1037 Vuotalo'],
 ['1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen',
  '1668 Kyläsaarenkatu 8, Stara toimipiste'],
 ['1603 Korttelitalo Merirasti', '1644 Työväen asuntomuseo'],
 ['1620 Tulli- ja pakkahuone, K 11', '1608 Koskelan ala-aste /Lpk Herttua'],
 ['1708 Pihlajamäen nuorisotalo', '1561 Työterveyskeskus']]

In [73]:
data['client_id'] = 0
data_new = pd.DataFrame(columns=data.columns)
for i in range(client_n):
    for j in range(cluster_n):
        tmp = data.loc[data['locationName'] == clients[i][j]]
        tmp['client_id'] = i
        data_new = pd.concat([data_new, tmp])
    
data_new

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,timestamp,locationName,value,client_id
0,2017-01-01 00:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",7.89,0
1,2017-01-01 01:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",7.90,0
2,2017-01-01 02:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",8.04,0
3,2017-01-01 03:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",7.93,0
4,2017-01-01 04:00:00,"1655 Lpk Virkkula, Ruiskumestarin talo",8.00,0
...,...,...,...,...
9795,2017-01-02 20:00:00,"4186 Lpk Pentinkulma, Lp Linnunrata",6.70,99
9796,2017-01-02 21:00:00,"4186 Lpk Pentinkulma, Lp Linnunrata",7.04,99
9797,2017-01-02 22:00:00,"4186 Lpk Pentinkulma, Lp Linnunrata",6.94,99
9798,2017-01-02 23:00:00,"4186 Lpk Pentinkulma, Lp Linnunrata",7.03,99


In [88]:
data_new.loc[data_new['client_id'] == 10].groupby('timestamp')['value'].apply(sum).to_frame()

,value
timestamp,
2017-01-01 00:00:00,47.82
2017-01-01 01:00:00,52.71
2017-01-01 02:00:00,52.01
2017-01-01 03:00:00,52.93
2017-01-01 04:00:00,52.31
2017-01-01 05:00:00,53.81
2017-01-01 06:00:00,54.49
2017-01-01 07:00:00,54.60
2017-01-01 08:00:00,52.62


In [93]:
# 每个client中的utility的总合
data_agg = pd.DataFrame(columns=data_new.columns)
for i in range(client_n):
    tmp = data_new.loc[data_new['client_id'] == i].groupby('timestamp')['value'].apply(sum).to_frame()
    tmp['client_id'] = i
    data_agg = pd.concat([data_agg, tmp])
data_agg = data_agg.drop(['timestamp', 'locationName'],axis=1)
data_agg

,value,client_id
2017-01-01 00:00:00,53.81,0
2017-01-01 01:00:00,52.70,0
2017-01-01 02:00:00,51.72,0
2017-01-01 03:00:00,53.37,0
2017-01-01 04:00:00,60.64,0
...,...,...
2017-01-02 20:00:00,8.29,99
2017-01-02 21:00:00,8.61,99
2017-01-02 22:00:00,8.53,99
2017-01-02 23:00:00,8.63,99
